# Feature engineering XGBoost

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [9]:
cols_a_eliminar = [
    "Unnamed: 0", "user_id",
]

fechas = [
    "fecha_primer_producto", "fecha_segundo_producto",
    "mes_mas_compras.x", "mes_mayor_monto.x"
]

binarias_explicit = ['checking_account', 'savings_account', 'credit_card', 'investment']

In [10]:
df = pd.read_csv("../data/interim/df_features_general.csv")
df.drop(columns=[col for col in cols_a_eliminar if col in df.columns], inplace=True)

In [11]:
# Convertir fechas
for col in fechas:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Convertir fechas y extraer timestamp  por requerimiento de sklearn
for col in fechas:
    if col in df.columns:
        # Convertir la columna a datetime
        df[col] = pd.to_datetime(df[col], errors='coerce')
        # >>> MODIFICACIÓN: Convertir fecha a formato numérico (timestamp en nanosegundos) usando astype
        df[col + "_ts"] = df[col].astype('int64')
        # Eliminar la columna original si no es necesaria
        df.drop(col, axis=1, inplace=True)

## Transfomaciones númericas

In [12]:
# Aplicar log1p para reducir asimetría de valores altos
if 'total_spend_fav' in df.columns:
    df['total_spend_fav'] = np.log1p(df['total_spend_fav'])

# Aplicar log1p para valores extremos en proporciones
if 'variacion_mensual_promedio_pct.x' in df.columns:
    df['variacion_mensual_promedio_pct.x'] = np.log1p(df['variacion_mensual_promedio_pct.x'])

# Winsorización de variable con outliers importantes
if 'variacion_mensual_promedio.x' in df.columns:
    p01 = df['variacion_mensual_promedio.x'].quantile(0.01)
    p99 = df['variacion_mensual_promedio.x'].quantile(0.99)
    df['variacion_mensual_promedio.x'] = df['variacion_mensual_promedio.x'].clip(p01, p99)

# Escalar recencia para estabilidad del gradiente
if 'recencia_transaccion' in df.columns:
    df['recencia_transaccion'] = StandardScaler().fit_transform(df[['recencia_transaccion']])

# Escalar otras variables numéricas continuas
escalar_xgb = [
    'age', 'dias_entre_productos', 'antiguedad_cliente', 'numero_productos',
    'entertainment_count', 'food_count', 'health_count', 'shopping_count',
    'supermarket_count', 'transport_count', 'travel_count'
]
escalar_xgb = [col for col in escalar_xgb if col in df.columns]
df[escalar_xgb] = StandardScaler().fit_transform(df[escalar_xgb])

## Tranformación categoricas label encoder

In [13]:
# Convertir variables binarias explícitas a tipo booleano
for col in binarias_explicit:
    if col in df.columns:
        df[col] = df[col].map({1: True, 0: False}).astype(bool)

# Codificación con LabelEncoder para categóricas (no binarias)
categoricas = [
    'income_range', 'risk_profile', 'occupation', 'age_range_sturges',
    'primer_producto', 'segundo_producto', 'combinacion_productos',
    'categoria_favorita_monto'
]
for col in categoricas:
    if col in df.columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))

In [15]:
print(df.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   age                               100 non-null    float64
 1   income_range                      100 non-null    int64  
 2   risk_profile                      100 non-null    int64  
 3   occupation                        100 non-null    int64  
 4   age_range_sturges                 100 non-null    int64  
 5   primer_producto                   100 non-null    int64  
 6   segundo_producto                  100 non-null    int64  
 7   dias_entre_productos              100 non-null    float64
 8   antiguedad_cliente                100 non-null    float64
 9   checking_account                  100 non-null    bool   
 10  savings_account                   100 non-null    bool   
 11  credit_card                       100 non-null    bool   
 12  insurance

## Dataset

In [14]:
y = df['insurance']
X = df.drop(columns=['insurance'])

# Guardar datasets preparados para modelado
X.to_csv("../data/processed/X_xgb_reduced.csv", index=False)
y.to_csv("../data/processed/y_xgb_reduced.csv", index=False)